In [2]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model, Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D, BatchNormalization, Dropout, Layer, LSTM, Input

In [3]:
dhw_merge = pd.read_csv('dhw_merge.csv')
elec_merge  = pd.read_csv('elec_merge.csv')
n_elec_merge = pd.read_csv('n_elec_merge.csv')

dhw_merge = dhw_merge.drop(['YEAR'], axis=1)
elec_merge = elec_merge.drop(['YEAR'], axis=1)
n_elec_merge = n_elec_merge.drop(['YEAR'], axis=1)

##  For Scaling

In [4]:
dhw_X = dhw_merge.drop(['DHW'], axis=1)
dhw_Y = dhw_merge[['DHW']]

scaler_dhw = MinMaxScaler()
scaler_dhw.fit(dhw_X)
scaled_dhw_X = scaler_dhw.transform(dhw_X)

new_dhw_X = pd.DataFrame(scaled_dhw_X, index=dhw_X.index, columns=dhw_X.columns)
new_dhw = pd.concat([new_dhw_X, dhw_Y], axis=1)

In [5]:
elec_X = elec_merge.drop(['ELEC'], axis=1)
elec_Y = elec_merge[['ELEC']]

scaler_elec = MinMaxScaler()
scaler_elec.fit(elec_X)
scaled_elec_X = scaler_elec.transform(elec_X)

new_elec_X = pd.DataFrame(scaled_elec_X, index=elec_X.index, columns=elec_X.columns)
new_elec = pd.concat([new_elec_X, elec_Y], axis=1)

In [6]:
n_elec_X = n_elec_merge.drop(['n_elec'], axis=1)
n_elec_Y = n_elec_merge[['n_elec']]

scaler_n_elec = MinMaxScaler()
scaler_n_elec.fit(n_elec_X)
scaled_n_elec_X = scaler_n_elec.transform(n_elec_X)

new_n_elec_X = pd.DataFrame(scaled_n_elec_X, index=n_elec_X.index, columns=n_elec_X.columns)
new_n_elec = pd.concat([new_n_elec_X, n_elec_Y], axis=1)

In [7]:
input_dhw = new_dhw.iloc[:-1,]
target_dhw = new_dhw[['DHW']].iloc[1:]

trainX_dhw, testX_dhw, trainY_dhw, testY_dhw = train_test_split(input_dhw,target_dhw,test_size=0.3,shuffle=False,random_state=0)

In [8]:
input_elec = new_elec.iloc[:-1,]
target_elec = new_elec[['ELEC']].iloc[1:]

trainX_elec, testX_elec, trainY_elec, testY_elec = train_test_split(input_elec,target_elec,test_size=0.3,shuffle=False,random_state=0)

In [9]:
input = new_n_elec.iloc[:-1,]
target = new_n_elec[['n_elec']].iloc[1:]

trainX_n_elec, testX_n_elec, trainY_n_elec, testY_n_elec = train_test_split(input,target,test_size=0.3,shuffle=False,random_state=0)

In [10]:
def buildDataSet(input, target, seqLength):
    xdata = []
    ydata = []
    for i in range(len(input) - seqLength):
        tx = input.iloc[i:i+seqLength]
        ty = target.iloc[i+seqLength-1]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)

In [11]:
trainx_dhw, trainy_dhw = buildDataSet(trainX_dhw, trainY_dhw, 3)
testx_dhw, testy_dhw = buildDataSet(testX_dhw, testY_dhw, 3)
trainx_elec, trainy_elec = buildDataSet(trainX_elec, trainY_elec, 3)
testx_elec, testy_elec = buildDataSet(testX_elec, testY_elec, 3)
trainx_n_elec, trainy_n_elec = buildDataSet(trainX_n_elec, trainY_n_elec, 3)
testx_n_elec, testy_n_elec = buildDataSet(testX_n_elec, testY_n_elec, 3)

## Reverse MinMaxScaler

In [12]:
# reverse_train_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_train_x)
# reverse_train_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_train_y)
# reverse_test_x = scaler_dhw_x_train.inverse_transform(scaled_dhw_test_x)
# reverse_test_y = scaler_dhw_y_train.inverse_transform(scaled_dhw_test_y)

# rev_dhw_train_x = pd.DataFrame(reverse_train_x, index=dhw_x_train.index, columns=dhw_x_train.columns)
# rev_dhw_train_y = pd.DataFrame(reverse_train_y, index=dhw_y_train.index, columns=dhw_y_train.columns)

# rev_dhw_test_x = pd.DataFrame(reverse_test_x, index=dhw_x_test.index, columns=dhw_x_test.columns)
# rev_dhw_test_y = pd.DataFrame(reverse_test_y, index=dhw_y_test.index, columns=dhw_y_test.columns)

# rev_dhw_trainSet = pd.concat([rev_dhw_train_x, rev_dhw_train_y], axis=1)
# rev_dhw_testSet = pd.concat([rev_dhw_test_x, rev_dhw_test_y], axis=1)

In [13]:
class GradientReversalLayer(Layer):
    def __init__(self, alpha=1.0):
        super(GradientReversalLayer, self).__init__()
        self.alpha = alpha
    
    def call(self, inputs):
        return self._reverse_gradient(inputs)
    
    def _reverse_gradient(self, x):
        return tf.negative(x) * self.alpha

class DANN(Model):
    def __init__(self, num_features, num_domains, hidden_units=64):
        super(DANN, self).__init__()

        # Feature Extractor
        self.feature_extractor = Sequential([
            LSTM(hidden_units, activation='swish', return_sequences=True),
            LSTM(hidden_units, activation='swish', return_sequences=False)
        ])
        
        # Domain Classifier
        self.domain_classifier = Sequential([
            Dense(hidden_units, activation='relu'),
            Dense(num_domains)  # Domain classifier predicts the domain label
        ])
        
        # Label Predictor
        self.label_predictor = Sequential([
            Dense(hidden_units, activation='relu'),
            Dense(1)  # For predicting energy consumption (or any other regression task)
        ])

        self.grl_layer = GradientReversalLayer()

    def call(self, inputs, training=False, alpha=1.0):
        features = self.feature_extractor(inputs)
        
        # Label Prediction
        label_prediction = self.label_predictor(features)
        
        if training:
            # Domain Prediction (using the gradient reversal layer)
            reversed_features = self.grl_layer(features, alpha=alpha)
            domain_prediction = self.domain_classifier(reversed_features)
        else:
            # During evaluation or inference, use the original features for domain prediction
            domain_prediction = self.domain_classifier(features)
        
        return label_prediction, domain_prediction

In [14]:
# Create an instance of the DANN model
num_features = 12  # Number of features in the input data
num_domains = 2    # Number of domains (source and target)

In [15]:
dann_model = DANN(num_features=num_features, num_domains=num_domains)

In [20]:
# 손실 함수 정의 (예시로 평균 제곱 오차 사용)
loss_object = tf.keras.losses.MeanSquaredError()

# 최적화를 위한 옵티마이저 정의 (예시로 Adam 사용)
optimizer = tf.keras.optimizers.Adam()

# EarlyStopping 콜백 생성
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # 모니터링할 성능 지표 (여기서는 검증 손실)
    patience=5,              # 성능이 향상되지 않더라도 몇 번까지 기다릴지 지정
    mode='min',              # 성능 지표가 최소화되어야 하는지 지정
    verbose=1                # 조기 종료가 적용될 때 메시지를 출력 (1: 출력, 0: 출력하지 않음)
)

# ModelCheckpoint 콜백 생성
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='model_weights.h5',  # 가중치 파일을 저장할 경로와 이름 지정
    save_best_only=True,          # 가장 좋은 성능을 보인 에포크의 가중치만 저장
    save_weights_only=True,       # 가중치만 저장 (아키텍처는 저장하지 않음)
    monitor='val_loss',           # 모니터링할 성능 지표 (여기서는 검증 손실)
    mode='min',                   # 성능 지표가 최소화되어야 하는지 지정
    verbose=1                     # 저장되는 과정을 출력 (1: 출력, 0: 출력하지 않음)
)

num_epochs = 10
alpha = 1.0

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # 기존에 가지고 있는 3차원 데이터 사용
    for i in range(0, len(trainX_n_elec), 1024):
        batch_x = trainX_n_elec[i:i + 1024]
        batch_y = trainY_n_elec[i:i + 1024]

        with tf.GradientTape() as tape:
            label_predictions, domain_predictions = dann_model(batch_x, training=True)
            # 손실 함수 계산
            label_loss = loss_object(batch_y, label_predictions)
            domain_loss = loss_object(1, domain_predictions)  # 도메인 레이블을 구현해야 합니다.
            total_loss = label_loss + domain_loss

        # 그래디언트 계산 및 가중치 업데이트
        gradients = tape.gradient(total_loss, dann_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, dann_model.trainable_variables))

    # # 검증 데이터를 사용하여 성능 평가
    # val_total_loss = 0.0
    # for i in range(0, len(val_data), batch_size):
    #     val_batch_x = val_data[i:i + batch_size]
    #     val_batch_y = val_labels[i:i + batch_size]

    #     label_predictions, domain_predictions = dann_model(val_batch_x, training=False, alpha=alpha)
    #     # 검증 결과 평가 (손실 함수 계산)
    #     val_label_loss = loss_object(val_batch_y, label_predictions)
    #     val_domain_loss = loss_object(도메인 레이블, domain_predictions)  # 도메인 레이블을 구현해야 합니다.
    #     val_total_loss += (val_label_loss + val_domain_loss)

    # average_val_loss = val_total_loss / num_val_batches
    # print(f"Epoch {epoch + 1}, Validation Loss: {average_val_loss.numpy():.4f}")

    # EarlyStopping 조건 충족 시 학습 중지
    if early_stopping_callback.stopped_epoch is not None:
        print("학습 조기 종료")
        break

# 최종 테스트 데이터에 대한 예측
predicted_values = dann_model(testx_n_elec, training=False, alpha=alpha)
# 예측 결과를 활용하여 분석 등 수행

Epoch 1/10
0     0.909091  0.000000  0.000000  0.455497  0.007634  0.000000  0.931034   
1     0.909091  0.000000  0.043478  0.462478  0.076336  0.444444  0.954023   
2     0.909091  0.000000  0.086957  0.462478  0.038168  0.388889  0.942529   
3     0.909091  0.000000  0.130435  0.460733  0.000000  0.000000  0.965517   
4     0.909091  0.000000  0.173913  0.464223  0.030534  0.000000  0.954023   
...        ...       ...       ...       ...       ...       ...       ...   
1019  1.000000  0.733333  0.956522  0.354276  0.015267  0.000000  1.000000   
1020  1.000000  0.733333  1.000000  0.361257  0.007634  0.000000  1.000000   
1021  1.000000  0.766667  0.000000  0.363002  0.053435  0.444444  1.000000   
1022  1.000000  0.766667  0.043478  0.366492  0.045802  0.555556  1.000000   
1023  1.000000  0.766667  0.086957  0.378709  0.045802  0.638889  1.000000   

            AP       SLP  VISIBILITY     GTEMP  n_elec  
0     0.710909  0.704174    0.213153  0.286331  6.1139  
1     0.707273  

ValueError: Exception encountered when calling layer "sequential" (type Sequential).

Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (1024, 12)

Call arguments received by layer "sequential" (type Sequential):
  • inputs=         MONTH       DAY      HOUR      TEMP        WS        WD       HUM  \
0     0.909091  0.000000  0.000000  0.455497  0.007634  0.000000  0.931034   
1     0.909091  0.000000  0.043478  0.462478  0.076336  0.444444  0.954023   
2     0.909091  0.000000  0.086957  0.462478  0.038168  0.388889  0.942529   
3     0.909091  0.000000  0.130435  0.460733  0.000000  0.000000  0.965517   
4     0.909091  0.000000  0.173913  0.464223  0.030534  0.000000  0.954023   
...        ...       ...       ...       ...       ...       ...       ...   
1019  1.000000  0.733333  0.956522  0.354276  0.015267  0.000000  1.000000   
1020  1.000000  0.733333  1.000000  0.361257  0.007634  0.000000  1.000000   
1021  1.000000  

In [ ]:
# 학습이 완료되면 테스트 데이터에 대한 예측값을 가져옵니다.
predicted_values = dann_model.predict(testx_n_elec)

In [ ]:
predicted_values.shape

In [ ]:
# Plot the predicted values against the actual values
plt.figure(figsize=(10, 6))
plt.plot(predicted_values, color='blue', label='Predicted')
plt.plot(testy_n_elec, color='red', linestyle='--', label='Actual')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
plt.legend()
plt.show()